# Dias Chuvosos

## Bibliotecas

In [10]:
import sys
sys.path.append('../src/') # Add src to path

In [43]:
import numpy as np
import pandas as pd
from glob import glob

In [38]:
from data.inmet import get_hour, get_timestamp, get_rainy_days

## Exemplo de dia chuvoso

In [53]:
data_dir = '../data/inmet'

csv_list = sorted(glob(f'{data_dir}/*.csv'))

print(f"Número de estações da região sul: {len(csv_list)}")
display(csv_list[0:5])

Número de estações da região sul: 95


['../data/inmet\\INMET_S_PR_A807_CURITIBA_01-01-2023_A_30-11-2023.CSV',
 '../data/inmet\\INMET_S_PR_A818_IVAI_01-01-2023_A_30-11-2023.CSV',
 '../data/inmet\\INMET_S_PR_A819_CASTRO_01-01-2023_A_30-11-2023.CSV',
 '../data/inmet\\INMET_S_PR_A820_MAL. CANDIDO RONDON_01-01-2023_A_30-11-2023.CSV',
 '../data/inmet\\INMET_S_PR_A821_JOAQUIM TAVORA_01-01-2023_A_30-11-2023.CSV']

In [29]:
estacao = csv_list[0] # Estação A807 - Curitiba

df = pd.read_csv(estacao,
                 encoding='iso-8859-1',
                 decimal=',',
                 delimiter=';',
                 skiprows=8,
                 usecols=['Data', 'Hora UTC', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'],
                 parse_dates=['Data'])

df.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)"
0,2023-01-01,0000 UTC,0.0
1,2023-01-01,0100 UTC,0.0
2,2023-01-01,0200 UTC,0.0
3,2023-01-01,0300 UTC,0.0
4,2023-01-01,0400 UTC,0.0


In [30]:
df['Data'] = df['Data'].astype(str)
df.rename(columns={"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": "precipitacao"}, inplace=True)

df['Hora'] = df['Hora UTC'].apply(get_hour)

df['timestamp'] = df[['Data', 'Hora']].apply(lambda x: get_timestamp(*x), axis=1)

df['timestamp'] = pd.to_datetime(df['timestamp'])

In [45]:
months_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] # Jan-Out

# Agrupar por dia (D), somando os valores de precipitação
df_result = (df.groupby(pd.Grouper(key='timestamp', axis=0, freq='D'))
             .sum(numeric_only=True)
             .sort_values(by=['precipitacao'], ascending=False)
             .reset_index()
             .copy())

# Filtra somente os meses em 'months_list'
df_result = df_result[df_result['timestamp']
                      .dt.month
                      .isin(months_list)
                      .copy()]

# Os dias mais chuvosos (mm/dia) da estação A807 - Curitiba
df_result.head(20)

,timestamp,precipitacao
1,2023-10-29,63.2
2,2023-10-12,58.0
3,2023-10-04,52.8
4,2023-01-04,48.0
5,2023-10-08,45.4
6,2023-10-07,38.4
9,2023-05-05,34.0
10,2023-10-17,31.2
11,2023-08-09,30.2
12,2023-08-25,29.2


## Lista de dias chuvosos

In [54]:
data_path = '../data/inmet'
months_list = list(np.arange(1, 11)) # Jan-Out
threshold = 50 # Somente precipitação (mm/dia) acima do limite

# Obtém os dias mais chuvosos
date_list = get_rainy_days(data_path, months_list, threshold)

print(f"Número de dias chuvosos (Jan-Out): {len(date_list)}")
display(date_list[0:5])

Número de dias chuvosos (Jan-Out): 107


['2023-01-02', '2023-01-03', '2023-01-10', '2023-01-12', '2023-01-13']

In [55]:
data_path = '../data/inmet'
months_list = [11] # Nov
threshold = 50 # Somente precipitação (mm/dia) acima do limite

# Obtém os dias mais chuvosos
date_list = get_rainy_days(data_path, months_list, threshold)

print(f"Número de dias chuvosos (Nov): {len(date_list)}")
display(date_list[0:5])

Número de dias chuvosos (Nov): 14


['2023-11-02', '2023-11-03', '2023-11-09', '2023-11-11', '2023-11-12']